In [25]:
# import libraries

import requests
import pandas as pd
import numpy as np
import json

def get_season_json(year_):
    
    year = year_
    league_id = 'league id here'
    url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
            str(league_id) + "?seasonId=" + str(year)
        
    r = requests.get(url, 
                     params={"view": "mMatchup"},
                     cookies={"swid": "{string here, retaining bracket and quotes}",
                              "espn_s2": "string here, retaining quotes"})
    d = r.json()[0]
    return d, league_id, url

def winner(x):
    if x>0:
        return 'W'
    if x==0:
        return 'Tie'
    else:
        return 'L'

def get_season_stats(year):    
    
    d, league_id, url = get_season_json(year)

    df = []
        
    for game in d['schedule']:

        ## omit week 14 and playoffs as some teams get wins via bye, messing up the win count

        if game['matchupPeriodId'] < 14:
            column = []
            column.append(game['matchupPeriodId'])
            column.append(game['home']['teamId']), 
            column.append(game['home']['totalPoints']),
            column.append(game['away']['teamId']), 
            column.append(game['away']['totalPoints'])
            df.append(column)

    df = pd.DataFrame(df, columns=['Week', 'Team1', 'Score1', 'Team2', 'Score2'])

    ## below is the original df generator from FFDP, which ran a KeyError because it couldn't find
    ## away teams in all matchups during bye weeks, so I replaced with that above
    
    # unroll json
    #df = [[
    #        game['matchupPeriodId'],
    #        game['home']['teamId'], game['home']['totalPoints'],
    #        game['away']['teamId'], game['away']['totalPoints']
    #    ] for game in d['schedule']] 
    #df = pd.DataFrame(df, columns=['Week', 'Team1', 'Score1', 'Team2', 'Score2'])
    
    ## didn't touch this
    # calculate margins and wins
    df3 = df.assign(Margin1 = df['Score1'] - df['Score2'],
                    Margin2 = df['Score2'] - df['Score1'])
    
    df3 = (df3[['Week', 'Team1', 'Margin1']]
     .rename(columns={'Team1': 'Team', 'Margin1': 'Margin'})
     .append(df3[['Week', 'Team2', 'Margin2']]
     .rename(columns={'Team2': 'Team', 'Margin2': 'Margin'})))
    
    df3['Win/Loss'] = df3['Margin'].apply(winner)
    wl = df3.groupby(['Team','Win/Loss']).count().reset_index()
    wl_tot = pd.pivot_table(wl, values='Week', index=['Team'],columns=['Win/Loss'], aggfunc=np.sum).reset_index()
    
    ## didn't touch this
    # calculate points for and against
    tm= []
    score = []
    against = []
  
    for i in df['Team1'].unique():

        x = (df[(df['Team1']==i)]['Score1'].sum() + df[(df['Team2']==i)]['Score2'].sum())#/len(df['Week'].unique())
        y = (df[(df['Team1']==i)]['Score2'].sum() + df[(df['Team2']==i)]['Score1'].sum())#/len(df['Week'].unique())
        tm.append(i)
        score.append(round(x,2))
        against.append(round(y,2))
    
    avg_scores = pd.DataFrame(list(zip(tm,score,against)),columns=['Team ID','Points For','Points Allowed'])
    
    ## added cookies here and removed the Pick Number-related language because it ran an error 
    ## and I didn't care much for seeing Pick Number to start
    # merge and add draft order
    league_his = avg_scores.merge(wl_tot,left_on='Team ID',right_on='Team')
    
    r = requests.get(url, 
                     params={"view": "mMatchup"},
                     cookies={"swid": "{string here, retaining bracket and quotes}",
                              "espn_s2": "string here, retaining quotes"})
    
    settings = r.json()[0]

    ## hid the following lines as explained above
    #l = settings['settings']['draftSettings']['pickOrder']
    #m = (np.arange(10)+1).tolist()
    
    league_his = league_his.sort_values(by='Team ID',ascending=True)
    #league_his['Pick Number'] = [x for _,x in sorted(zip(l,m))]
    league_his['Year']=year
   

    ## removed Pick Number as a column below
    if 'Tie' in league_his.columns:
        return league_his[['Team ID','W','L','Tie','Points For','Points Allowed','Year']].sort_values(by='W',ascending=False)
    else:
        return league_his[['Team ID','W','L','Points For','Points Allowed','Year']].sort_values(by='W',ascending=False)

    
## created this function

def get_historicals(year_):
    frames = []
    year = year_
    while year < 2021:
        frames.append(get_season_stats(year))
        year = year + 1
    
    final_table = pd.concat(frames)
    
    res = final_table.groupby('Team ID', as_index=False).sum()

    a = res.iloc[:,:5].sort_values(by='W',ascending=False)
    print(a)

In [26]:
get_historicals(2017)

   Team ID   W   L  Points For  Points Allowed
3        4  38  14     6062.00         5105.86
1        2  33  19     6219.36         5472.58
8        9  30  22     5547.58         5306.04
2        3  29  23     5713.24         5353.28
0        1  28  24     5593.50         5615.68
5        6  27  25     5607.26         5477.58
4        5  23  29     5303.92         5531.80
9       10  19  33     4963.82         5438.58
6        7  17  35     5058.28         5877.38
7        8  16  36     4605.94         5496.12
